Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

In [0]:
import sys
import pandas as pd
# import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [0]:
%%capture
# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

[data documentation](https://baseballsavant.mlb.com/csv-docs)

In [0]:
sample = 'https://raw.githubusercontent.com/michael-rowland/Pitch-Predictions/master/sample.csv'
df = pd.read_csv(sample)

In [41]:
initial_features = ['pitch_name', 'game_date', 'description', 'zone', 'des', 
                    'stand', 'type', 'bb_type', 'balls', 'strikes', 'on_3b', 
                    'on_2b', 'on_1b', 'outs_when_up', 'inning', 'at_bat_number',
                    'pitch_number', 'bat_score', 'fld_score']

df = df[initial_features]
df = df[df['game_date'] == '2018-09-28']
df.head()

,pitch_name,game_date,description,zone,des,stand,type,bb_type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,at_bat_number,pitch_number,bat_score,fld_score
0,4-Seam Fastball,2018-09-28,hit_into_play,4.0,Adam Engel pops out to second baseman Gregorio...,R,X,popup,2,2,NaN,NaN,NaN,2,7,53,5,1,2
1,Curveball,2018-09-28,ball,14.0,NaN,R,B,NaN,1,2,NaN,NaN,NaN,2,7,53,4,1,2
2,Curveball,2018-09-28,ball,14.0,NaN,R,B,NaN,0,2,NaN,NaN,NaN,2,7,53,3,1,2
3,4-Seam Fastball,2018-09-28,swinging_strike,5.0,NaN,R,S,NaN,0,1,NaN,NaN,NaN,2,7,53,2,1,2
4,Curveball,2018-09-28,called_strike,9.0,NaN,R,S,NaN,0,0,NaN,NaN,NaN,2,7,53,1,1,2


In [0]:
def pitch_counter(df):

  df = df.copy()
  
  # I THINK THIS SHOULD BE ABLE TO BE REPLACED WITH .rolling
  df['4fastball_count'] = 0
  df['2fastball_count'] = 0
  df['curveball_count'] = 0
  df['changeup_count'] = 0

  four_fastball_counter = 0
  two_fastball_counter = 0
  curveball_counter = 0
  changeup_counter = 0

  for i in range(len(df)):
    if df.loc[i, 'previous_pitch'] == '4-Seam Fastball':
      four_fastball_counter += 1
    df['4fastball_count'].iloc[i] = four_fastball_counter

    if df.loc[i, 'previous_pitch'] == '2-Seam Fastball':
      two_fastball_counter += 1
    df['2fastball_count'].iloc[i] = two_fastball_counter

    if df.loc[i, 'previous_pitch'] == 'Curveball':
      curveball_counter += 1
    df['curveball_count'].iloc[i] = curveball_counter

    if df.loc[i, 'previous_pitch'] == 'Changeup':
      changeup_counter += 1
    df['changeup_count'].iloc[i] = changeup_counter
    
  return df

In [0]:
def pitch_tendency(df, window):

  df = df.copy()
  
  # USE AN ENCODER HERE
  df['4fastball'] = (df['previous_pitch'] == '4-Seam Fastball') * 1
  df['2fastball'] = (df['previous_pitch'] == '2-Seam Fastball') * 1
  df['curveball'] = (df['previous_pitch'] == 'Curveball') * 1
  df['changeup'] = (df['previous_pitch'] == 'Changeup') * 1

  df[f'4fastball_tendency_{window}'] = df['4fastball'].rolling(
      window, min_periods=1).mean()
  df[f'2fastball_tendency_{window}'] = df['2fastball'].rolling(
      window, min_periods=1).mean()
  df[f'curveball_tendency_{window}'] = df['curveball'].rolling(
      window, min_periods=1).mean()
  df[f'changeup_tendency_{window}'] = df['changeup'].rolling(
      window, min_periods=1).mean()

  return df

In [75]:
def wrangle(df):
  
  df = df.copy()
  
  # CALCULATES SCORE MARGIN
  df['score_margin'] = df['fld_score'] - df['bat_score']
  df = df.drop(['fld_score', 'bat_score'], axis=1)

  # CALCULATES TOTAL PITCHES THROWN SINCE ENTERING GAME
  df['pitch_total'] = df.groupby('game_date').cumcount(ascending=False)+1

  # MAPS 1b/2b/3b TO BOOLEAN
  df['on_1b'] = df['on_1b'].notnull()
  df['on_2b'] = df['on_2b'].notnull()
  df['on_3b'] = df['on_3b'].notnull()

  # CREATES 'at_bat_count' FROM 'at_bat_number'
  '''This probably needs to be updated when I update apply groupby function for date'''
  df['at_bat_count'] = df.groupby('at_bat_number').ngroup(ascending=True)+1
  df = df.drop('at_bat_number', axis=1)

  # CREATES 'pitcher_advantage' COLUMN
  df['count'] = df['balls'].astype(str) + '-' + df['strikes'].astype(str)
  df['pitcher_advantage'] = df['count'].map({'0-0':'neutral', '0-1':'ahead', 
                                             '0-2':'ahead', '1-0':'neutral', 
                                             '1-1':'neutral', '1-2':'ahead', 
                                             '2-0':'behind', '2-1':'neutral', 
                                             '2-2':'ahead', '3-0':'behind', 
                                             '3-1':'behind', '3-2':'neutral'})
  df = df.drop('count', axis=1)
  

  # PITCH TENDENCIES
  df['previous_pitch'] = df['pitch_name'].shift(1)
  df = pitch_counter(df)
  df = pitch_tendency(df, 5)
  
  return df

output = wrangle(df)
output.T

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
pitch_name,4-Seam Fastball,Curveball,Curveball,4-Seam Fastball,Curveball,2-Seam Fastball,2-Seam Fastball,Curveball,Curveball,Curveball,2-Seam Fastball,Curveball,4-Seam Fastball,4-Seam Fastball,2-Seam Fastball,4-Seam Fastball,Curveball,4-Seam Fastball,Curveball,4-Seam Fastball,Changeup,2-Seam Fastball,Curveball,2-Seam Fastball,2-Seam Fastball,Curveball,Curveball,Curveball,2-Seam Fastball,2-Seam Fastball,4-Seam Fastball,4-Seam Fastball,Curveball,Curveball,2-Seam Fastball,4-Seam Fastball,Curveball,Curveball,4-Seam Fastball,4-Seam Fastball,Curveball,Curveball,2-Seam Fastball,Curveball,2-Seam Fastball,4-Seam Fastball,4-Seam Fastball,4-Seam Fastball,2-Seam Fastball,Curveball,2-Seam Fastball,2-Seam Fastball,Changeup,4-Seam Fastball,4-Seam Fastball,Changeup,2-Seam Fastball,2-Seam Fastball,Curveball,2-Seam Fastball,Changeup,2-Seam Fastball,Curveball,4-Seam Fastball,4-Seam Fastball,2-Seam Fastball,Curveball,4-Seam Fastball,Curveball,2-Seam Fastball,2-Seam Fastball,Curveball,Curveball,4-Seam Fastball,2-Seam Fastball,Curveball,4-Seam Fastball,2-Seam Fastball,4-Seam Fastball,2-Seam Fastball,4-Seam Fastball,Curveball,Curveball,4-Seam Fastball,4-Seam Fastball,2-Seam Fastball,2-Seam Fastball,2-Seam Fastball,Curveball,2-Seam Fastball,2-Seam Fastball,Curveball,2-Seam Fastball,4-Seam Fastball,Curveball,Curveball,4-Seam Fastball,Curveball,4-Seam Fastball,Curveball,4-Seam Fastball,4-Seam Fastball,4-Seam Fastball,4-Seam Fastball,Changeup,4-Seam Fastball
game_date,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28,2018-09-28
description,hit_into_play,ball,ball,swinging_strike,called_strike,hit_into_play,ball,ball,ball,swinging_strike,foul,ball,swinging_strike,swinging_strike,called_strike,swinging_strike,called_strike,foul,called_strike,ball,swinging_strike,called_strike,ball,ball,called_strike,ball,ball,hit_into_play,foul,swinging_strike,foul,swinging_strike,foul,swinging_strike_blocked,foul,foul,hit_into_play,ball,foul,called_strike,hit_into_play,ball,called_strike,ball,ball,called_strike,ball,ball,ball,swinging_strike,foul,called_strike,ball,ball,foul,swinging_strike,ball,ball,hit_into_play,called_strike,hit_into_play_score,ball,ball,foul,foul,foul,hit_into_play_no_out,swinging_strike,called_strike,ball,hit_into_play,ball,ball,called_strike,ball,hit_into_play,called_strike,called_strike,hit_into_play_no_out,foul,ball,called_strike,ball,hit_into_play,foul,swinging_strike,hit_into_play,ball,called_strike,ball,ball,called_strike,hit_into_play,swinging_strike,swinging_strike,ball,ball,swinging_strike,called_strike,ball,called_strike,called_strike,called_strik

In [38]:
output['pitch_name'].value_counts()

Curveball          36
4-Seam Fastball    34
2-Seam Fastball    31
Changeup            5
Name: pitch_name, dtype: int64

In [7]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, minimal=True).to_notebook_iframe()

profile

In [15]:
pd.__version__

'0.25.3'